# Period Module

## Initialisation

Import modules, etc

In [1]:
import unittest

from common import Printable, projdir

from name import Name
from fuzzy import FuzzyMatch
from entrant import Entrant

from constants import *

## Period Class

The period class contains common methods for events, sessions and courses

In [2]:
class Period(Printable):
    def __init__(self, parent=None, verbosity=1):
        '''Initialise period object'''

        super().__init__(verbosity=verbosity)

        self.parent = parent

        if parent:
            self.entrants = parent.entrants
            self.names = parent.names
            self.sailNos = parent.sailNos
            self.gt31Ids = parent.gt31Ids
            self.fuzzyMatch = parent.fuzzyMatch
            self.appConfig = parent.appConfig

        else:
            self.entrants = {}
            self.names = {}
            self.sailNos = {}
            self.gt31Ids = {}
            self.fuzzyMatch = FuzzyMatch()
            self.appConfig = None

        self.runs = {}
        self.numRuns = 0


    def getEntrantBySailNo(self, sailNo, name):
        '''Get entrant from the sail number'''

        if sailNo not in self.sailNos:

            # Start by attempting a quick lookup of the name itself
            # TODO - match by craft type as well as name
            if name and name in self.names:
                entrant = self.names[name][0]
                self.sailNos[sailNo] = entrant
                self.logWarning('Auto-matched sail number {} to {} ({})'.format(
                    sailNo, entrant.getName(), entrant.getValue('Craft Type')))
               
            else:
                # Next try looking for entrant names that count as a fuzzy match
                # TODO - match by craft type as well as name
                entrants = []
                if name:
                    nameObj = Name(name)
                    for entrantId in self.entrants:
                        entrant = self.entrants[entrantId]
                        if self.fuzzyMatch.matchNameObjects(entrant.name, nameObj):
                            entrants.append(entrant)

                # Only accept a unique match - multiple matches will be ignored
                if len(entrants) == 1:
                    entrant = entrants[0]
                    self.names[name] = [entrant]
                    self.sailNos[sailNo] = entrant
                    self.logWarning('Auto-matched sail number {} to {} ({})'.format(
                        sailNo, entrant.getName(), entrant.getValue('Craft Type')))

                else:
                    entrantId = max(self.entrants) + 1

                    if name:
                        entrant = Entrant(["ID", "Sail Number", "Name"], [entrantId, sailNo, name], verbosity=self.verbosity)
                        self.logWarning('Unrecognised sail number {} ({}) on {}'.format(sailNo, entrant.getName(), self.date))
                    else:
                        entrant = Entrant(["ID", "Sail Number"], [entrantId, sailNo], verbosity=self.verbosity)
                        self.logWarning('Unrecognised sail number {} on {}'.format(sailNo, self.date))

                    self.entrants[entrantId] = entrant
                    self.names[name] = [entrant]
                    self.sailNos[sailNo] = entrant

        else:
            entrant = self.sailNos[sailNo]

            # TODO - check craft type as well as name
            if name not in self.names:
                if name and name != entrant.getName():
                    nameObj = Name(name)
                    if self.fuzzyMatch.matchNameObjects(nameObj, entrant.name) is False:
                        self.logWarning('Name mismatch for sail {} - {} vs {}'.format(sailNo, name, entrant.getName()))

                self.names[name] = [entrant]

        return entrant


    def getEntrantByGt31(self, gt31Id, gt31Serial):
        '''Get entrant from the GT-31 ID and serial'''

        if gt31Id not in self.gt31Ids:
            self.logWarning('Unrecognised GT-31 ID on {} - {}'.format(self.date, gt31Id))

            entrantId = max(self.entrants) + 1
            entrant = Entrant(["ID", "GT31 ID", "GT31 SN"], [entrantId, gt31Id, gt31Serial], verbosity=self.verbosity)
            name = entrant.getName()

            self.entrants[entrantId] = entrant
            self.names[name] = [entrant]
            self.gt31Ids[gt31Id] = entrant

        # TODO - check for unique serial (just in case GPS name has been changed)

        else:
            entrant = self.gt31Ids[gt31Id]

            # Only report unrecognised GT-31 serials if the entrant had any GT-31 serials registered
            if entrant.gt31SerialNumbers and gt31Serial not in entrant.gt31SerialNumbers:
                self.logWarning('Unrecognised GT-31 SN for {} ({}) on {} - {} vs {}'.format(
                        entrant.getValue('Name'), entrant.getValue('Craft Type'), self.date,
                        gt31Serial, entrant.gt31SerialNumbers))

                entrant.gt31SerialNumbers.add(gt31Serial)

        return entrant


    def storeRun(self, entrantId, speedRun):
        '''Store run in memory'''

        if entrantId in self.runs:
            self.runs[entrantId].append(speedRun)
        else:
            self.runs[entrantId] = [speedRun]
            
        self.numRuns += 1

        if self.parent:
            self.parent.storeRun(entrantId, speedRun)


    def sortRuns(self):
        '''Sort runs for each person, fastest to slowest'''

        for entrantId in self.runs:
            self.runs[entrantId].sort(key=lambda x: x.data[T_SPEED], reverse=True)

## Unit Tests

A handful of very basic tests, including a dummy session class

In [3]:
class TestPeriod(unittest.TestCase):
    '''Class to test Period class'''
    
    def testDummy(self, session=None):
        '''Test using dummy data'''

        pass

## Run Unit Tests

Note: Only run unit tests when running this script directly, not during an import

In [4]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


## All Done!